In [4]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch, os

# Load pre-trained model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Function for text summarization
def summarize_text(input_text, max_length=1000): # Adjust max_length as needed
    input_ids = tokenizer.encode(input_text, return_tensors='pt', max_length=1024, truncation=True)
    summary_ids = model.generate(input_ids, max_length=max_length, min_length=135, length_penalty=2.0, num_beams=4, early_stopping=True) # Adjust min_length as needed (originally 50)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Function to extract 'Past Medical History' section from file ending with '-First.txt'
def extract_past_medical_history_first():
    first_file = None
    # Find the first file ending with '-First.txt.'
    for filename in os.listdir():
        if filename.endswith('-First.txt'):
            first_file = filename
            break
    if first_file:
        with open(first_file, 'r') as file:
            lines = file.readlines()
            past_medical_history_section = []
            start_index = None
            for line in lines:
                if 'Past Medical History:' in line:
                    start_index = True
                elif start_index:
                    if line.strip():  # Check for empty lines
                        past_medical_history_section.append(line.strip())
                    else:
                        break
            # Adding label for past medical history section
            return 'Past Medical History:\n' + '\n'.join(past_medical_history_section)
    else:
        return ""

# Function to extract 'Follow-up' section from file ending with '-Last.txt'
def extract_follow_up():
    last_file = None
    # Find the last file ending with '-Last.txt.'
    for filename in os.listdir():
        if filename.endswith('-Last.txt'):
            last_file = filename
    if last_file:
        with open(last_file, 'r') as file:
            lines = file.readlines()
            follow_up_section = []
            start_index = None
            for i, line in enumerate(lines):
                if 'Follow-up:' in line:
                    start_index = i + 1
                elif start_index is not None:
                    if line.strip():  # Check for empty lines
                        follow_up_section.append(line.strip())
                    else:
                        break
            return '\n'.join(follow_up_section)
    else:
        return ""

# List to store text from all files
all_text = []

# Iterate over files in the current directory
print('Iterating over files in current directory... \n \n')
for filename in os.listdir():
    if filename.endswith('.txt'):
        with open(filename, 'r') as file:
            # Read content of the text file
            text_content = file.read()
            # Append text to list
            all_text.append(text_content)

# Combine text from all files into a single string
print('Combining all the individual text files... \n \n')
combined_text = '\n'.join(all_text)

# Extract 'Past Medical History' section from the first file ending with '-First.txt' and append to combined_text
past_medical_history_section = extract_past_medical_history_first()
if past_medical_history_section:
    combined_text += '\n\n' + past_medical_history_section

# Extract 'Follow-up' section from the last file ending with '-Last.txt' and append to combined_text
follow_up_section = extract_follow_up()

if follow_up_section:
    combined_text += '\n\n' + follow_up_section

# Summarise combined text
print('Summarising combined text... \n \n')
summary = summarize_text(combined_text)

# Format the summary output with spaces between categories
summary = summary.replace('.', '.\n')

# Display the combined summary
print("Combined Summary:\n", summary)

Iterating over files in current directory... 
 

Combining all the individual text files... 
 

Summarising combined text... 
 

Combined Summary:
 Patient Name: Jane Smith.
Date of Admission: 2024-03-10.
Gender: Female.
 Chief Complaint: Persistent cough and shortness of breath.
 Medications: Albuterol inhaler as needed, Lisinopril 10 mg daily, Amlodipine 5 mg daily.
 Physical Examination: Blood Pressure: 140/90 mmHg, Heart Rate: 85 bpm, Respiratory Rate: 20 bpm.
 Oxygen Saturation: 88% on room air, 86% on air.
 Increased wheezing and crackles on auscultation.
 Follow-up: Consult pulmonology for further evaluation and treatment.

